In [36]:
from astroquery.vizier import Vizier

from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time
import astropy.units as u

import pandas as pd
import numpy as np

import datetime

from IPython.display import Markdown, HTML

## User Input

In [45]:
# Specify the catalog identifier (the bibcode)
catalog_id = "II/183A"  # This is the VizieR catalog ID for the paper


## Get the Catalog Table

In [46]:
# Initialize the Vizier object
vizier = Vizier(row_limit=-1)

# Retrieve the table(s)
tables = vizier.get_catalogs(catalog_id)

# Check if any tables were found
if tables:
    # The 'tables' object is an AstroPy TableList.
    # You can access individual tables by index (e.g., tables[0])
    # or by the table name if available.

    # Print the first table to inspect its contents
    #print(tables[0])

    # You can access the data as an AstroPy Table object
    data_table = tables[0]
    df = data_table.to_pandas()
else:
    print(f"No tables found for catalog ID: {catalog_id}")
    
df[0:5]

,RAJ2000,DEJ2000,Vmag,e_Vmag,B-V,e_B-V,U-B,e_U-B,V-R,e_V-R,R-I,e_R-I,V-I,e_V-I,SimbadName
0,00 30 09,-46 31 22,14.651,0.0028,0.793,0.0046,0.380,0.0071,0.435,0.0019,0.405,0.0035,0.841,0.0032,[L92b] TPHE A
1,00 30 16,-46 27 55,12.334,0.0115,0.405,0.0026,0.156,0.0039,0.262,0.0020,0.271,0.0019,0.535,0.0035,[L92b] TPHE B
2,00 30 17,-46 32 34,14.376,0.0022,-0.298,0.0024,-1.217,0.0043,-0.148,0.0038,-0.211,0.0133,-0.360,0.0149,[L92b] TPHE C
3,00 30 18,-46 31 11,13.118,0.0033,1.551,0.0030,1.871,0.0118,0.849,0.0015,0.810,0.0023,1.663,0.0030,[L92b] TPHE D
4,00 30 19,-46 24 36,11.630,0.0017,0.443,0.0012,-0.103,0.0024,0.276,0.0007,0.283,0.0015,0.564,0.0019,[L92b] TPHE E


In [47]:
ra_list  = df["RAJ2000"].astype(str).tolist()
dec_list = df["DEJ2000"].astype(str).tolist()
coords   = SkyCoord(ra=ra_list, dec=dec_list, unit=(u.degree,u.degree))

try:
    df.insert(2, 'RA_hourangle', coords.ra.value)
    df.insert(3, 'DEC_deg', coords.dec.value)

except ValueError:
    print('RA (in hourangle) and DEC (in deg) already exist in Landolt catalogue database')
    


In [48]:
try:    
    df.insert(4, 'Bmag', df["B-V"] + df["Vmag"])
    df.insert(5, 'e_Bmag', np.sqrt(df["e_B-V"]**2 + df["e_Vmag"]**2))
    
    df.insert(8, 'Rmag', df["Vmag"] - df["V-R"])
    df.insert(9, 'e_Rmag', np.sqrt(df["e_Vmag"]**2 + df["e_V-R"]**2))
    
    df.insert(10, 'Imag', df["Rmag"] - df["R-I"])
    df.insert(11, 'e_Imag', np.sqrt(df["e_Rmag"]**2 + df["e_R-I"]**2))
    
except ValueError:
    print('The VRI columns already exist in the Landolt catalogue dataframe')
df[0:5]

,RAJ2000,DEJ2000,RA_hourangle,DEC_deg,Bmag,e_Bmag,Vmag,e_Vmag,Rmag,e_Rmag,...,e_B-V,U-B,e_U-B,V-R,e_V-R,R-I,e_R-I,V-I,e_V-I,SimbadName
0,00 30 09,-46 31 22,0.502500,-46.522778,15.444000,0.005385,14.651,0.0028,14.216,0.003384,...,0.0046,0.380,0.0071,0.435,0.0019,0.405,0.0035,0.841,0.0032,[L92b] TPHE A
1,00 30 16,-46 27 55,0.504444,-46.465278,12.738999,0.011790,12.334,0.0115,12.072,0.011673,...,0.0026,0.156,0.0039,0.262,0.0020,0.271,0.0019,0.535,0.0035,[L92b] TPHE B
2,00 30 17,-46 32 34,0.504722,-46.542778,14.078000,0.003256,14.376,0.0022,14.524,0.004391,...,0.0024,-1.217,0.0043,-0.148,0.0038,-0.211,0.0133,-0.360,0.0149,[L92b] TPHE C
3,00 30 18,-46 31 11,0.505000,-46.519722,14.669000,0.004460,13.118,0.0033,12.269,0.003625,...,0.0030,1.871,0.0118,0.849,0.0015,0.810,0.0023,1.663,0.0030,[L92b] TPHE D
4,00 30 19,-46 24 36,0.505278,-46.410000,12.073000,0.002081,11.630,0.0017,11.354,0.001838,...,0.0012,-0.103,0.0024,0.276,0.0007,0.283,0.0015,0.564,0.0019,[L92b] TPHE E


### Selects stars with User Criteria

In [49]:
df.columns

Index(['RAJ2000', 'DEJ2000', 'RA_hourangle', 'DEC_deg', 'Bmag', 'e_Bmag',
       'Vmag', 'e_Vmag', 'Rmag', 'e_Rmag', 'Imag', 'e_Imag', 'B-V', 'e_B-V',
       'U-B', 'e_U-B', 'V-R', 'e_V-R', 'R-I', 'e_R-I', 'V-I', 'e_V-I',
       'SimbadName'],
      dtype='object')

In [58]:
# criteria
list_mag      = [9,11]
tolerance_mag = 0.5 



# Location
latitude = -6.824631  * u.deg  # Latitude of Bosscha Obs
longitude = 107.617111 * u.deg
elevation = 1100 * u.m 
earth_location = EarthLocation(lat=latitude, lon=longitude , height=elevation)

now_utc = Time(datetime.datetime.utcnow(), scale='utc', location=earth_location)
lst = now_utc.sidereal_time('apparent')
lst_hour = round(lst.hour,0)

In [59]:
l_title = True
columns       = ["SimbadName",'RAJ2000', 'DEJ2000', 'Vmag', 'e_Vmag','B-V', 'e_B-V']
for mag in list_mag:
    
    v_mask  = (df['Vmag'] >= mag - tolerance_mag) & (df['Vmag'] <= mag + tolerance_mag)
    RA_mask = (df['RA_hourangle'] >= lst_hour - 2) & (df['RA_hourangle'] <= lst_hour + 2)
    
    if l_title:
        display(Markdown(f"### List Bintang V ~ {mag} dengan {lst_hour-2} < RA(J2000) < {lst_hour+2}"))
        l_title = False
    else:
        display(Markdown(f"### List Bintang V ~ {mag}"))
    markdown_table = df[RA_mask & v_mask][columns].to_markdown(index=False)
    display(Markdown(markdown_table))
    

### List Bintang V ~ 9 dengan 15.0 < RA(J2000) < 19.0

| SimbadName   | RAJ2000   | DEJ2000   |   Vmag |   e_Vmag |   B-V |   e_B-V |
|:-------------|:----------|:----------|-------:|---------:|------:|--------:|
| SA 109 231   | 17 45 20  | -00 25 51 |  9.332 |   0.0011 | 1.462 |  0.0014 |

### List Bintang V ~ 11

| SimbadName   | RAJ2000   | DEJ2000   |   Vmag |   e_Vmag |   B-V |   e_B-V |
|:-------------|:----------|:----------|-------:|---------:|------:|--------:|
| SA 107 970   | 15 37 25  | +00 18 34 | 10.939 |   0.0074 | 1.596 |  0.0017 |
| SA 107 484   | 15 40 17  | -00 21 13 | 11.311 |   0.0044 | 1.237 |  0.0022 |
| SA 108 475   | 16 37 00  | -00 34 40 | 11.309 |   0.0014 | 1.38  |  0.0013 |
| SA 108 551   | 16 37 47  | -00 33 06 | 10.703 |   0.0011 | 0.179 |  0.0007 |
| SA 108 1918  | 16 37 50  | -00 00 37 | 11.384 |   0.0007 | 1.432 |  0.0014 |
| SA 109 71    | 17 44 06  | -00 24 59 | 11.493 |   0.001  | 0.323 |  0.0011 |
| SA 109 199   | 17 45 02  | -00 29 28 | 10.99  |   0.0098 | 1.739 |  0.004  |
| SA 110 506   | 18 43 19  | +00 30 27 | 11.312 |   0.0021 | 0.568 |  0.0021 |
| SA 110 441   | 18 43 34  | +00 19 40 | 11.121 |   0.0015 | 0.555 |  0.0015 |

In [60]:
l_title = True
columns       = ["SimbadName",'RAJ2000', 'DEJ2000', 'Bmag', 'e_Bmag','B-V', 'e_B-V']
for mag in list_mag:
    
    v_mask  = (df['Bmag'] >= mag - tolerance_mag) & (df['Bmag'] <= mag + tolerance_mag)
    RA_mask = (df['RA_hourangle'] >= lst_hour - 2) & (df['RA_hourangle'] <= lst_hour + 2)
    
    if l_title:
        display(Markdown(f"### List Bintang B ~ {mag} dengan {lst_hour-2} < RA(J2000) < {lst_hour+2}"))
        l_title = False
    else:
        display(Markdown(f"### List Bintang B ~ {mag}"))
    markdown_table = df[RA_mask & v_mask][columns].to_markdown(index=False)
    display(Markdown(markdown_table))

### List Bintang B ~ 9 dengan 15.0 < RA(J2000) < 19.0

| SimbadName   | RAJ2000   | DEJ2000   | Bmag   | e_Bmag   | B-V   | e_B-V   |
|--------------|-----------|-----------|--------|----------|-------|---------|

### List Bintang B ~ 11

| SimbadName   | RAJ2000   | DEJ2000   |   Bmag |     e_Bmag |   B-V |   e_B-V |
|:-------------|:----------|:----------|-------:|-----------:|------:|--------:|
| SA 108 551   | 16 37 47  | -00 33 06 | 10.882 | 0.00130384 | 0.179 |  0.0007 |
| BD-04 4226   | 17 05 15  | -05 05 05 | 11.486 | 0.00351141 | 1.415 |  0.0012 |
| SA 109 231   | 17 45 20  | -00 25 51 | 10.794 | 0.00178045 | 1.462 |  0.0014 |
| SA 109 537   | 17 45 42  | -00 21 34 | 10.962 | 0.00295466 | 0.609 |  0.0012 |

In [61]:
l_title = True
columns       = ["SimbadName",'RAJ2000', 'DEJ2000', 'Rmag', 'e_Rmag', 'Vmag', 'V-R', 'e_V-R']
for mag in list_mag:
    
    v_mask  = (df['Rmag'] >= mag - tolerance_mag) & (df['Rmag'] <= mag + tolerance_mag)
    RA_mask = (df['RA_hourangle'] >= lst_hour - 2) & (df['RA_hourangle'] <= lst_hour + 2)
    
    if l_title:
        display(Markdown(f"### List Bintang R ~ {mag} dengan {lst_hour-2} < RA(J2000) < {lst_hour+2}"))
        l_title = False
    else:
        display(Markdown(f"### List Bintang R ~ {mag}"))
    markdown_table = df[RA_mask & v_mask][columns].to_markdown(index=False)
    display(Markdown(markdown_table))

### List Bintang R ~ 9 dengan 15.0 < RA(J2000) < 19.0

| SimbadName   | RAJ2000   | DEJ2000   |   Rmag |     e_Rmag |   Vmag |   V-R |   e_V-R |
|:-------------|:----------|:----------|-------:|-----------:|-------:|------:|--------:|
| BD-12 4523   | 16 30 18  | -12 39 08 |  8.917 | 0.00327567 | 10.069 | 1.152 |  0.0007 |
| BD-04 4226   | 17 05 15  | -05 05 05 |  9.101 | 0.00332415 | 10.071 | 0.97  |  0.0004 |
| SA 109 231   | 17 45 20  | -00 25 51 |  8.547 | 0.001253   |  9.332 | 0.785 |  0.0006 |

### List Bintang R ~ 11

| SimbadName   | RAJ2000   | DEJ2000   |   Rmag |     e_Rmag |   Vmag |   V-R |   e_V-R |
|:-------------|:----------|:----------|-------:|-----------:|-------:|------:|--------:|
| SA 107 1006  | 15 38 34  | +00 14 22 | 11.27  | 0.00128062 | 11.712 | 0.442 |  0.0008 |
| SA 107 592   | 15 38 51  | -00 17 10 | 11.138 | 0.00354683 | 11.847 | 0.709 |  0.0013 |
| SA 107 458   | 15 38 50  | -00 24 27 | 11.009 | 0.00504777 | 11.676 | 0.667 |  0.0028 |
| SA 107 484   | 15 40 17  | -00 21 13 | 10.647 | 0.0047927  | 11.311 | 0.664 |  0.0019 |
| SA 108 1848  | 16 36 58  | +00 05 57 | 11.407 | 0          | 11.738 | 0.331 |  0      |
| SA 108 475   | 16 37 00  | -00 34 40 | 10.565 | 0.00161245 | 11.309 | 0.744 |  0.0008 |
| SA 108 551   | 16 37 47  | -00 33 06 | 10.604 | 0.001253   | 10.703 | 0.099 |  0.0006 |
| SA 108 1918  | 16 37 50  | -00 00 37 | 10.611 | 0.00425793 | 11.384 | 0.773 |  0.0042 |
| WOLF 629     | 16 55 27  | -08 18 53 | 10.574 | 0.00440114 | 11.759 | 1.185 |  0.0016 |
| SA 109 71    | 17 44 06  | -00 24 59 | 11.307 | 0.00128062 | 11.493 | 0.186 |  0.0008 |
| SA 109 381   | 17 44 12  | -00 20 32 | 11.302 | 0.00142127 | 11.73  | 0.428 |  0.0009 |
| SA 110 355   | 18 42 19  | +00 08 24 | 11.292 | 0.00348281 | 11.944 | 0.652 |  0.0022 |
| SA 110 266   | 18 42 49  | +00 05 06 | 11.48  | 0.0019799  | 12.018 | 0.538 |  0.0014 |
| SA 110 499   | 18 43 07  | +00 28 00 | 11.137 | 0.00363593 | 11.737 | 0.6   |  0.0019 |
| SA 110 502   | 18 43 10  | +00 27 40 | 10.957 | 0.00363593 | 12.33  | 1.373 |  0.0019 |
| SA 110 503   | 18 43 11  | +00 29 43 | 11.4   | 0.00358469 | 11.773 | 0.373 |  0.0018 |
| SA 110 506   | 18 43 19  | +00 30 27 | 10.977 | 0.00221359 | 11.312 | 0.335 |  0.0007 |
| SA 110 290   | 18 43 22  | -00 01 15 | 11.48  | 0.00370135 | 11.898 | 0.418 |  0.0023 |
| SA 110 441   | 18 43 34  | +00 19 40 | 10.797 | 0.00192094 | 11.121 | 0.324 |  0.0012 |
| SA 110 450   | 18 43 52  | +00 22 58 | 11.033 | 0.00228254 | 11.585 | 0.552 |  0.0011 |
| SA 110 319   | 18 43 55  | +00 02 01 | 11.119 | 0.0117047  | 11.861 | 0.742 |  0.004  |

In [62]:
l_title = True
columns       = ["SimbadName",'RAJ2000', 'DEJ2000', 'Imag', 'e_Imag','Vmag', 'Rmag', 'R-I', 'e_R-I']
for mag in list_mag:
    
    v_mask  = (df['Imag'] >= mag - tolerance_mag) & (df['Imag'] <= mag + tolerance_mag)
    RA_mask = (df['RA_hourangle'] >= lst_hour - 2) & (df['RA_hourangle'] <= lst_hour + 2)
    
    if l_title:
        display(Markdown(f"### List Bintang I ~ {mag} dengan {lst_hour-2} < RA(J2000) < {lst_hour+2}"))
        l_title = False
    else:
        display(Markdown(f"### List Bintang I ~ {mag}"))
    markdown_table = df[RA_mask & v_mask][columns].to_markdown(index=False)
    display(Markdown(markdown_table))

### List Bintang I ~ 9 dengan 15.0 < RA(J2000) < 19.0

| SimbadName   | RAJ2000   | DEJ2000   |   Imag |    e_Imag |   Vmag |   Rmag |   R-I |   e_R-I |
|:-------------|:----------|:----------|-------:|----------:|-------:|-------:|------:|--------:|
| WOLF 629     | 16 55 27  | -08 18 53 |  9.049 | 0.0045618 | 11.759 | 10.574 | 1.525 |  0.0012 |
| SA 109 199   | 17 45 02  | -00 29 28 |  9.084 | 0.0111485 | 10.99  |  9.984 | 0.9   |  0.004  |

### List Bintang I ~ 11

| SimbadName         | RAJ2000   | DEJ2000   |   Imag |     e_Imag |   Vmag |   Rmag |   R-I |   e_R-I |
|:-------------------|:----------|:----------|-------:|-----------:|-------:|-------:|------:|--------:|
| [L92b] PG1528+062B | 15 30 40  | +06 01 11 | 11.281 | 0          | 11.989 | 11.625 | 0.344 |  0      |
| SA 107 1006        | 15 38 34  | +00 14 22 | 10.849 | 0.00156525 | 11.712 | 11.27  | 0.421 |  0.0009 |
| SA 107 459         | 15 38 51  | -00 22 31 | 11.242 | 0.0032078  | 12.284 | 11.759 | 0.517 |  0.0007 |
| SA 107 602         | 15 39 19  | -00 15 29 | 11.04  | 0.0037921  | 12.116 | 11.571 | 0.531 |  0.0022 |
| SA 108 1848        | 16 36 58  | +00 05 57 | 11.082 | 0          | 11.738 | 11.407 | 0.325 |  0      |
| SA 108 1863        | 16 37 13  | +00 02 34 | 11.4   | 0          | 12.244 | 11.798 | 0.398 |  0      |
| SA 108 981         | 16 39 17  | -00 25 08 | 11.449 | 0.00438748 | 12.071 | 11.761 | 0.312 |  0.003  |
| SA 109 71          | 17 44 06  | -00 24 59 | 11.084 | 0.00150997 | 11.493 | 11.307 | 0.223 |  0.0008 |
| SA 109 381         | 17 44 12  | -00 20 32 | 10.867 | 0.00163095 | 11.73  | 11.302 | 0.435 |  0.0008 |
| SA 109 954         | 17 44 15  | -00 02 17 | 10.941 | 0.00983209 | 12.436 | 11.672 | 0.731 |  0.0027 |
| SA 110 229         | 18 40 45  | +00 01 51 | 11.296 | 0.00410488 | 13.649 | 12.451 | 1.155 |  0.002  |
| SA 110 233         | 18 40 52  | +00 00 51 | 11.18  | 0.00380789 | 12.771 | 11.998 | 0.818 |  0.0015 |
| SA 110 355         | 18 42 19  | +00 08 24 | 10.565 | 0.00411947 | 11.944 | 11.292 | 0.727 |  0.0022 |
| SA 110 266         | 18 42 49  | +00 05 06 | 10.903 | 0.00288617 | 12.018 | 11.48  | 0.577 |  0.0021 |
| SA 110 365         | 18 42 57  | +00 07 20 | 10.84  | 0.00406694 | 13.47  | 12.11  | 1.27  |  0.0022 |
| SA 110 157         | 18 42 57  | -00 08 56 | 11.095 | 0.00824985 | 13.491 | 12.234 | 1.139 |  0.0049 |
| SA 110 280         | 18 43 07  | -00 03 40 | 10.613 | 0.00487852 | 12.996 | 11.761 | 1.148 |  0.003  |
| SA 110 503         | 18 43 11  | +00 29 43 | 10.964 | 0.00392556 | 11.773 | 11.4   | 0.436 |  0.0016 |
| SA 110 506         | 18 43 19  | +00 30 27 | 10.665 | 0.00474763 | 11.312 | 10.977 | 0.312 |  0.0042 |
| SA 110 507         | 18 43 19  | +00 29 26 | 11.228 | 0.00645368 | 12.44  | 11.807 | 0.579 |  0      |
| SA 110 290         | 18 43 22  | -00 01 15 | 11.062 | 0.00470213 | 11.898 | 11.48  | 0.418 |  0.0029 |
| SA 110 315         | 18 43 52  | +00 00 49 | 11.298 | 0.00936056 | 13.637 | 12.431 | 1.133 |  0.0057 |